In [159]:
import os
import random
import warnings

import pandas as pd
import spacy
from dotenv import load_dotenv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sqlalchemy import create_engine

load_dotenv()
warnings.filterwarnings("ignore")

In [160]:
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DBNAME = os.getenv("POSTGRES_DBNAME")

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

data = pd.read_sql_query("SELECT * FROM get_book_data", cnx)
data.fillna("missing", inplace=True)
data.shape

(52, 11)

In [161]:
data.iloc[random.choices(range(len(data)), k=10)]

,id,book_title,language,publisher_name,category_1,category_2,category_3,author_1,author_2,short_description,full_description
34,97,Quán trọ Ancuta,Tiếng việt,Dân Trí,Kinh điển,Nước ngoài,Tiểu thuyết,Mihail Sadoveanu,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>
20,83,Sách Giáo Khoa Hoá Học Lớp 10,Tiếng việt,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>
12,75,Naruto - Tập 1,Tiếng việt,NXB Kim Đồng,Giả tưởng,Manga,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>
6,67,Thám tử lừng danh Conan - Tập 104,Tiếng việt,NXB Kim Đồng,Manga,Trinh thám,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>
18,81,Sách Giáo Khoa Khoa Học Lớp 5,Tiếng việt,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod a

In [162]:
nlp = spacy.blank("vi")
nlp.Defaults.stop_words.add("sách")


def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


data["cleaned_text"] = (
        data["book_title"] + " " +
        data["publisher_name"] + " " +
        data["category_1"] + " " +
        data["category_2"] + " " +
        data["category_3"] +
        data["author_1"] + " " +
        data["author_2"] + " " +
        data["short_description"] + " " +
        data["full_description"] + " "
)

data["cleaned_text"] = data["cleaned_text"].apply(preprocess_text)

data.iloc[random.choices(range(len(data)), k=10)]

,id,book_title,language,publisher_name,category_1,category_2,category_3,author_1,author_2,short_description,full_description,cleaned_text
15,78,Sách Giáo Khoa Toán Lớp 5,Tiếng việt,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>,giáo khoa toán lớp 5 giáo dục việt nam giáo khoa missing missingmissing missing < p > dolor labore incididunt eiusmod magna labore nbsp dipiscing ipsum dolore amet eiusmod aliqua et < p > < h1 > sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore < h1 > \r\n < p > elit sit lorem aliqua et et et eiusmod sed eiusmod dolor elit tempor ipsum elit eiusmod amet dolor magna aliqua elit magna labore eiusmod incididunt elit tempor amet consectetur tempor adipiscing < p > \r\n < p > dolore ipsum magna amet consectetur eiusmod elit ipsum et labore amet adipiscing elit dolor consectetur aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt magna eiusmod elit ipsum lorem sed ut lorem sit dolore labore sit ipsum ut sit elit elit sed < p > \r\n < p > nbsp < p >
31,94,Thần thoại Hy Lạp,Tiếng việt,NXB Trẻ,Tiểu thuyết,Trong nước,missing,Nguyễn Văn Khoa,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet eiusmod aliqua et.</p>,<h1>Sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore.</h1>\r\n<p>Elit sit lorem aliqua et et et eiusmod sed eiusmod dolor. Elit tempor ipsum elit eiusmod amet dolor magna. Aliqua elit magna labore eiusmod incididunt elit tempor. Amet consectetur tempor adipiscing do.</p>\r\n<p>Dolore ipsum magna do amet consectetur eiusmod elit ipsum et labore amet adipiscing do. Elit dolor consectetur do aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur. Adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt. Magna eiusmod elit ipsum lorem sed ut lorem sit. Dolore labore sit ipsum ut sit elit elit sed.</p>\r\n<p>&nbsp;</p>,thần thoại hy lạp nxb trẻ tiểu thuyết missingnguyễn văn khoa missing < p > dolor labore incididunt eiusmod magna labore nbsp dipiscing ipsum dolore amet eiusmod aliqua et < p > < h1 > sed elit dolor aliqua tempor lorem sit sit tempor lorem incididunt incididunt labore < h1 > \r\n < p > elit sit lorem aliqua et et et eiusmod sed eiusmod dolor elit tempor ipsum elit eiusmod amet dolor magna aliqua elit magna labore eiusmod incididunt elit tempor amet consectetur tempor adipiscing < p > \r\n < p > dolore ipsum magna amet consectetur eiusmod elit ipsum et labore amet adipiscing elit dolor consectetur aliqua incididunt labore dolore dolore dolore incididunt sed magna consectetur adipiscing tempor ipsum sit eiusmod sed incididunt dolor et sit labore dolor incididunt magna eiusmod elit ipsum lorem sed ut lorem sit dolore labore sit ipsum ut sit elit elit sed < p > \r\n < p > nbsp < p >
47,111,Tiểu thuyết Naruto - Itachi chân truyền - Ám dạ thiên,Tiếng việt,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing,<p>Dolor labore incididunt eiusmod magna labore.&nbsp;Dipiscing ipsum dolore amet

In [163]:
cv = CountVectorizer()
clean_description = cv.fit_transform(data['cleaned_text']).toarray()
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(clean_description)

NearestNeighbors(algorithm='brute', metric='cosine')

In [169]:
def recommend(user_input):
    clean_user_input = cv.transform([preprocess_text(user_input)]).toarray()
    distances, indices = model.kneighbors(clean_user_input, n_neighbors=5)
    return data.iloc[indices[0]][["id", "book_title"]].to_dict(orient="records")

In [170]:
user_description = "tôi muốn mua sách tiểu thuyết của nxb kim đồng"

recommend(user_description)

[{'id': 111,
  'book_title': 'Tiểu thuyết Naruto - Itachi chân truyền - Ám dạ thiên'},
 {'id': 112,
  'book_title': 'Tiểu thuyết Naruto - Itachi chân truyền - Quang minh thiên'},
 {'id': 115, 'book_title': 'Gambit Hậu'},
 {'id': 87, 'book_title': 'Viên ngọc trai kì diệu'},
 {'id': 93, 'book_title': 'Tiểu Thuyết Naruto - Sasuke Chân Truyền'}]